In [125]:
import gym
from gym import spaces
import numpy as np
from math import floor

class TrafficGenerator(gym.Env):
    
    def encode(self, mean_delay_q1, mean_delay_q2, mean_delay_q3):
        mean_delay_q1 = -floor(mean_delay_q1)
        mean_delay_q2 = -floor(mean_delay_q2)
        mean_delay_q3 = -floor(mean_delay_q3)
        encoded_queue = mean_delay_q1*100 + mean_delay_q2*10 + mean_delay_q3
        return encoded_queue

    def decode(self, encoded_queue):
        mean_delay_q3 = encoded_queue % 10
        mean_delay_q2 = (encoded_queue // 10) % 10
        mean_delay_q1 = encoded_queue // 100
        return [mean_delay_q1, mean_delay_q2, mean_delay_q3]
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        self.observation_space = spaces.Discrete(1000)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.35, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    print("Appending to queue", i)
        
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
                
        
        # services the queues
        for i in range(len(self.queues)):
            print("queue ", i, ": has a length of", len(self.queues[i]))
        print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)
            for i in range(len(self.queues[action])):
                self.queues[action][i] -= 1
        

        
        #TODO rewardoptions?
        

        
         # Loop over the packet types
        observation = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
    
            observation.append(current_waiting_time)
            print(current_waiting_time)

#                 print(reward)
#                 print(self.curr_mean_delay_best_effort)
#                 print(current_waiting_time)
        if observation[0] < -4:
            reward = -10
        elif observation[1] < -6:
            reward = -10
        else:
            reward = 1
        observation = self.encode(observation[0], observation[1], observation[2])
        done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0
        return observation, done, reward, {}
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        q = np.array([[len(self.queues[i]), 0.0] for i in range(len(self.mean_delay_req))])
        return self.encode(0, 0, 0)
        
    def render(self):
        pass

In [128]:
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
penalties = 0
while x < 100:
    print("~~~~~~~~~")
    action = env.action_space.sample()
    
    print("chosen", action)
    obs, done, reward, _ = env.step(action)
    if reward == -10:
        penalties += 1
    x = x + 1

print("Total Penalties: " + str(penalties))


~~~~~~~~~
chosen 1
Appending to queue 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 1
ACTION queue  1 : has a length of 0
0.0
0.0
0.0
~~~~~~~~~
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 1
ACTION queue  2 : has a length of 1
0.0
0.0
0.0
~~~~~~~~~
chosen 0
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  0 : has a length of 0
0.0
0.0
0.0
~~~~~~~~~
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
~~~~~~~~~
chosen 0
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  0 : has a length of 0
0.0
0.0
0.0
~~~~~~~~~
chosen 0
Appending to queue 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 1
ACTION queue  0 : has a length of 0
0.0
0.0
0.0
~~~~~~~~~
chosen 0
Appending to qu

In [129]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])


In [133]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 1000):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        
        print("chosen", action)

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")
print(f"Total Penalties: {penalties}")


Episode: 900
chosen 1
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  1 : has a length of 0
0.0
0.0
0.0
chosen 1
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  1 : has a length of 0
0.0
0.0
0.0
chosen 1
Appending to queue 0
Appending to queue 1
queue  0 : has a length of 1
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  1 : has a length of 1
0.0
0.0
0.0
chosen 1
Appending to queue 0
Appending to queue 1
queue  0 : has a length of 1
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  1 : has a length of 1
0.0
0.0
0.0
chosen 1
Appending to queue 0
Appending to queue 1
queue  0 : has a length of 1
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  1 : has a length of 1
0.0
0.0
0.0
chosen 1
Appending to queue 0
queue  0 : has a length of 1
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  1 : has a 

0.0
0.0
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
chosen 2
Appending to queue 1
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
chosen 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
chosen 2
Appending to queue 1
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a le

In [134]:

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Appending to queue 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 1
ACTION queue  2 : has a length of 1
0.0
0.0
0.0
Appending to queue 1
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
Appending to queue 0
queue  0 : has a length of 1
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
Appending to queue 2
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 1
ACTION queue  2 : has a length of 1
0.0
0.0
0.0
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
queue  0 : has a length of 0
queue  1 : has a length of 0
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
0.0
0.0
0.0
Appending to queue 1
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a length of 